教程地址：https://medium.com/data-science/object-based-land-cover-classification-with-python-cbe54e9c9e24
对应视频地址：https://www.youtube.com/playlist?list=PLzHdTn7Pdxs6R6gf-0aLCqy8pL1GazPEe
我感觉你飘了，先跟着教程跑一遍啊。

In [1]:

import numpy as np
from osgeo import gdal
from skimage.segmentation import slic
import os
import ospybook as pb
from skimage import io
folder = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Landsat\Utah'
raster_fns = ['LE70380322000181EDC02_60m.tif', 'LE70380322000181EDC02_TIR_60m.tif']
os.chdir(folder)
driverTiff = gdal.GetDriverByName('GTiff')
band_data = pb.stack_bands(raster_fns)

img = band_data

In [2]:
ds = gdal.Open(raster_fns[0])
print(type(ds.GetRasterBand(1).ReadAsArray()))

<class 'numpy.ndarray'>


In [ ]:
# 疑问：这些块是要和原本图片差不多更重要，还是说分成很多小块更重要？
# 答：根据教程来看，分得小并且这些小块组成的大块与原本图片相同更为重要。
segments = slic(img, n_segments=25, compactness=0.2) 
io.imshow(segments)

In [ ]:
import scipy
def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
    return features

In [ ]:
segment_ids = np.unique(segments)
objects = []
object_ids = []
for id in segment_ids:
    segment_pixels = img[segments == id]
    object_features = segment_features(segment_pixels)
    objects.append(object_features)
    object_ids.append(id)